In [1]:
import pandas as pd 
import numpy as np
import glob
import os
import math
import scipy.stats as stats
from scipy.stats import pearsonr
from sklearn.decomposition import FactorAnalysis
import pingouin as pg
import semopy
import statsmodels.api as sm
from sklearn import linear_model
from mlxtend.feature_selection import SequentialFeatureSelector

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Read CSV
tam_df = pd.read_csv('tam_results/tam.csv')
demographics = pd.read_csv('demographic_results/demographics_processed.csv')

In [3]:
# Merge
tam_df = pd.merge(tam_df, demographics[['user_id', 'user_archetype', 'age', 'sex_new', 'medical_speciality_new', 'grade_new', 'ai_familiarity_new']])
# Move columns 
col = tam_df.pop('q28')
tam_df.insert(23, col.name, col)
col = tam_df.pop('q29')
tam_df.insert(24, col.name, col)
'''# To int
tam_df['q28'] = tam_df['q28'].fillna(np.nan)
tam_df['q29'] = tam_df['q29'].fillna(np.nan)
tam_df['q28'] = tam_df['q28'].astype(int)
tam_df['q29'] = tam_df['q29'].astype(int)'''
# Update column names
#AA_1
tam_df.columns = ['user_id',
 'user_archetype',
 'ai_attitude',
 'PU1',
 'PU2',
 'PU3',
 'PU4',
 'PU5',
 'PU6',
 'PU7',
 'PU8',
 'PEOU1',
 'PU9',
 'PEOU2',
 'SE1',
 'SE2',
 'voluntary',
 'SN1',
 'SN2',
 'prestige_image',
 'PU10',
 'PU11',
 'BC1',
 'BC2',
 'BC3',
 'clinical_workflow',
 'infrastructure',
 'intention',
 'colleagues',
 'PU12',
 'final_comments',
 'age_new',
 'sex_new',
 'medical_speciality_new',
 'grade_new',
 'ai_familiarity_new']

 # Get result columns 
tam_df_results = tam_df.iloc[:,2:30]

"# To int\ntam_df['q28'] = tam_df['q28'].fillna(np.nan)\ntam_df['q29'] = tam_df['q29'].fillna(np.nan)\ntam_df['q28'] = tam_df['q28'].astype(int)\ntam_df['q29'] = tam_df['q29'].astype(int)"

In [12]:
# Get groups
tam_groups = ['PU', 'PEOU', 'SE', 'SN', 'BC']

In [23]:
# Mean and SD
for group in tam_groups:
    group_df = tam_df_results.filter(regex=group)
    print(group)
    print(group_df.mean().mean())
    print(group_df.mean().std())

PU
3.591269841269842
0.3084118109246874
PEOU
3.8333333333333335
0.20203050891044205
SE
4.059523809523809
0.05050762722761036
SN
3.2023809523809526
0.11785113019775781
BC
3.46031746031746
0.6967585027731611


In [15]:
# Cronbach's alpha
for group in tam_groups:
    group_df = tam_df_results.filter(regex=group)
    if group_df.shape[1] > 1:
        cronbach_alpha = pg.cronbach_alpha(group_df)
        print(group_df.columns)
        print(f"Cronbach's alpha for {group}:", cronbach_alpha)

Index(['PU1', 'PU2', 'PU3', 'PU4', 'PU5', 'PU6', 'PU7', 'PU8', 'PU9', 'PU10',
       'PU11', 'PU12'],
      dtype='object')
Cronbach's alpha for PU: (0.8442116498764807, array([0.764, 0.906]))
Index(['PEOU1', 'PEOU2'], dtype='object')
Cronbach's alpha for PEOU: (0.5162659123055162, array([0.1 , 0.74]))
Index(['SE1', 'SE2'], dtype='object')
Cronbach's alpha for SE: (0.5137039937353172, array([0.095, 0.739]))
Index(['SN1', 'SN2'], dtype='object')
Cronbach's alpha for SN: (0.07116382505559682, array([-0.728,  0.501]))
Index(['BC1', 'BC2', 'BC3'], dtype='object')
Cronbach's alpha for BC: (0.39662947865265, array([-0.007,  0.655]))
